# MACHINE LEARNING NÃO SUPERVISIONADO - CLUSTERING

Este projeto tem por objetivo desenvolver um algoritmo de Machine Learning para agrupar clientes do shopping.

os dados foram extraidos do Kaggle: 
https://www.kaggle.com/shwetabh123/mall-customers

In [5]:
# Importando as bibliotecas
import pandas as pd
import numpy as np

In [6]:
# Importando a biblioteca "warnings" para ignorar mensagens de erro
import warnings
warnings.filterwarnings("ignore")

In [7]:
# Importando o arquivo CSV e criando um dataframe
df = pd.read_csv("/content/drive/MyDrive/MACHINE_LEARNNING/Mall_Customers.csv", sep=",", encoding="iso-8859-1")

In [8]:
# Visualizando as 5 primeiras linhas do dataframe
df.head()

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


# ATRIBUTOS:
* CustomerID: Identificação do cliente
* Genre: Gênero
* Age: Idade
* Annual Income (k$): Rendimento anual
* Spending Score (1-100): Pontuação de gastos

# Visualização gráfica dos dados

## Idade

In [9]:
# Importando o Plotly para criar representações gráficas dos atributos
import plotly.express as px

In [10]:
# Criando um histograma com o atributo Age ( Idade )
hist = px.histogram( df, x = "Age", nbins = 60)
hist.update_layout (width = 600, height = 400, title_text = "Distribuição de Idades")
hist.show()

O gráfico mostra uma distribuição diversificada entre as idades, sendo a 
menor idade 18 anos com 4 contagens e a idade máxima 70 anos com 2 contagens.

## Gênero

In [11]:
# Agora veremos as distribuições de gênero
hist = px.histogram( df, x = "Genre", nbins = 60)
hist.update_layout (width = 600, height = 400, title_text = "Distribuição de gênero")
hist.show()
df["Genre"].value_counts() # contagem por tipo

Female    112
Male       88
Name: Genre, dtype: int64

In [12]:
# Visualizando a quantidade de linhas e colunas
df.shape

(200, 5)

# Exploração e tratamento dos dados

## Alterando o nome das colunas

In [13]:
df.head()

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [14]:
# Alterando o nome da coluna Genre para Gênero
df.rename(columns={"Genre":"genero"},inplace=True)

In [15]:
# Alterando o nome da coluna Age para Idade
df.rename(columns={"Age":"idade"},inplace=True)

In [16]:
# Alterando o nome da coluna Annual Income (k$) para Rendimento
df.rename(columns={"Annual Income (k$)":"rendimento"},inplace=True)

In [17]:
# Alterando o nome da coluna Spending Score (1-100) para Pontuação
df.rename(columns={"Spending Score (1-100)":"pontuacao"},inplace=True)

In [18]:
df.head()

,CustomerID,genero,idade,rendimento,pontuacao
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [19]:
# Obs.: não alterei a coluna CustomerID porque ela será irrelevante e será excluída

## Valores Missing ( NAN )

In [20]:
df.isnull().sum()

CustomerID    0
genero        0
idade         0
rendimento    0
pontuacao     0
dtype: int64

Verificamos que não existem valores nulos.

## Análise dos tipos de atributos:
* object = string
* int64 = números inteiros
* float64 = números reais
* complex = números complexos


Obs.: Lembrando que não é possivel desenvolver modelos de machine learning usando atributos do tipo OBJECT

In [21]:
df.dtypes

CustomerID     int64
genero        object
idade          int64
rendimento     int64
pontuacao      int64
dtype: object

## Dados estatísticos

In [22]:
df.describe()

,CustomerID,idade,rendimento,pontuacao
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,38.850000,60.560000,50.200000
std,57.879185,13.969007,26.264721,25.823522
min,1.000000,18.000000,15.000000,1.000000
25%,50.750000,28.750000,41.500000,34.750000
50%,100.500000,36.000000,61.500000,50.000000
75%,150.250000,49.000000,78.000000,73.000000
max,200.000000,70.000000,137.000000,99.000000


Podemos verificar que:
* A amostra possui 200 elementos
* A média de idade dos frequentadores do shopping fica em 38, e a mediana 36
* A idade mínima é 18 e a máxima é 70
* O rendimento máximo encontrado foi de $137000
* A pontuação média fica em 50.2

## Análise de Outliers ( Dados discrepantes )

Para essa análise utilizamos o gráfico boxplot

In [23]:
# Importando o Plotly para criar representações gráficas dos atributos
import plotly.express as px

In [24]:
boxplot = px.box( df, y = "rendimento")
boxplot.show()

Obs.: Rendimentos possui um valor fora do padrão de 137 mil, acima do valor máximo de 129 mil. Por não ser um valor tão distante podemos manter no modelo.

In [25]:
boxplot = px.box( df, y = "idade")
boxplot.show()

In [26]:
boxplot = px.box( df, y = "pontuacao")
boxplot.show()

Nenhum dado discrepante encontrado nos outros atributos.

# Pré-processamento

O pré-processamento é uma etapa fundamental que pode melhorar a performance dos algoritmos de análise, através da redução de dimensionalidade e eliminação de ruidos que interfiram no funcionamento dos algoritmos. 